##### 이론
1. 0~5값은 연속적인 정보를 포함함. -> 7이상 값 NaN등으로 처리 필요
2. 예, 아니오는 범주형 칼럼 -> 자료형을 str로 변환해 7 이상값도 하나의 범주로 볼 수 있게끔 처리

- dra_01z1 평생음주여부
- drb_01z3 연간음주빈도
    - 결측치 처리: 연간 음주 빈도의 1번 칼럼값 = 평생 음주 빈도의 2번 칼럼값
    - 가설1: 음주를 자주 할수록 흡연자 비율이 높아질 것이다 (완료)

- drb_03z1 한 번 섭취 시 음주량
    - 가설2: 음주량이 많을 수록 흡연자 비율이 높아질 것이다 (완료)

- drb_04z1 월간 폭음 경험(남)
- drb_05z1 월간 폭음 경험(여)
    - 가설3: 폭음 경험이 잦을수록 흡연자 비율이 높아질 것이다 (완료)
    - 성별에 대해 '월간 폭음 경험' 칼럼으로 새 피쳐 생성 필요

- drg_01z3 절주 또는 금주계획 여부
    - 가설4: 절주 또는 금주를 잘 실천하고 있을수록 금연자 비율이 높을 것이다 (완료)

- 음주폐해 예방 또는 절주에 대한 홍보or교육 경험 여부 drop


- pha_04z1 고강도 신체활동 일수
- pha_07z1 중강도 신체활동 일수
- phb_01z1 걷기 실천 일수
    - 결측치 처리. NaN
    - 가설5: 신체활동을 자주하는 사람일수록 금연자일 비율이 높을 것이다 (미미함)

- pha_11z1 최근 1주일 유연성 운동 실천
    - 가설6: 유연성 운동을 자주 실천하는 사람일수록 금연자 비율이 높을 것이다 (완료)

In [95]:
# 활용할 칼럼의 값 설명

# 평생 음주 여부 - 범례형
dra_01z1 = {
    1: '예',
    2: '아니오',
    7: '응답거부'
}

# 연간 음주 빈도 - 커질수록 과음
drb_01z3 = {
    1:'최근 1년간 전혀 마시지 않았다',
    2:'한 달에 1번 미만',
    3:'한 달에 1번 정도',
    4:'한 달에 2~4번 정도',
    5:'일주일에 2~3번 정도',
    6:'일주일에 4번 이상',
    7:'응답거부',
    8:'비해당',
    9:'모름'
}

# 한 번 섭취 시 음주량 - 커질수록 과음
drb_03z1 = {
    1:'1~2잔',
    2:'3~4잔',
    3:'5~6잔',
    4:'7~9잔',
    5:'10잔 이상',
    7:'응답거부',
    8:'비해당',
    9:'모름'
}

# 월간 폭음 경험 - 커질수록 과음
drb_binge_monthly = {
    1:'전혀 없다',
    2:'한 달에 1번 미만',
    3:'한 달에 1번 정도',
    4:'일주일에 1번 정도',
    5:'거의 매일',
    7:'응답거부',
    8:'비해당',
    9:'모름'
}

# 절주 또는 금주계획 여부 - 커질수록 긍정적 (절주나 금주 실천중)
drg_01z3 = {
    1: '현재로서는 전혀 절주 또는 금주할 생각이 없다',
    2: '향후 6개월 이내에 절주 또는 금주할 계획이 있다',
    3: '향후 1개월 이내에 절주 또는 금주할 계획이 있다',
    4: '현재 절주 또는 금주를 실천하고 있으며, 그 기간이 6개월 미만이다',
    5: '현재 절주 또는 금주를 실천하고 있으며, 그 기간이 6개월 이상이다',
    7:'응답거부',
    8:'비해당',
    9:'모름'
}

# 고강도, 중강도, 걷기 일수: 일주일 중 _일 - 커질수록 긍정적 (자주 운동)
#   77. 응답거부, 99. 모름

# 최근 1주일 유연성 운동 실천 - 커질수록 긍정적 (자주 운동)
pha_11z1 = {
    1:'전혀 하지 않음',
    2: '1일',
    3: '2일',
    3: '3일',
    4: '4일',
    5: '5일 이상',
    9: '모름'
}

In [96]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib
import seaborn as sns
from scipy.stats import pointbiserialr

font_path = 'C:/Windows/Fonts/gulim.ttc'
font = fm.FontProperties(fname=font_path).get_name()
matplotlib.rc('font', family=font) #폰트 설정 변경
# plt.rcParams['axes.unicode_minus']=False #마이너스 기호 깨질 때


import warnings
warnings.filterwarnings('ignore')
#warning 메세지 무시 코드(출력 안 됨)

In [97]:
# 데이터 로드
df = pd.read_csv('../../data/analy_data.csv')

In [98]:
columns = {
    'dra_01z1':	'평생 음주 여부',
    'drb_01z3':	'연간 음주 빈도',
    'drb_03z1':	'한 번 섭취 시 음주량',
    'drb_04z1':	'월간 폭음 경험(남)',
    'drb_05z1':	'월간 폭음 경험(여)',
    'drg_01z3':	'절주 또는 금주계획 여부',
    'pha_04z1':	'고강도 신체활동 일수',
    'pha_07z1':	'중강도 신체활동 일수',
    'phb_01z1':	'걷기 실천 일수',
    'pha_11z1':	'최근 1주일 유연성 운동 실천',
    'churn': '금연 성공 여부'
}

dr_ph_columns = list(columns.keys())

dr_ph_df = df[dr_ph_columns]

### 음주 칼럼 select 과정

In [99]:
# 1. '평생 음주 여부', '연간 음주 빈도' ['dra_01z1', 'drb_01z3']

# 1-1. 평생 음주 여부 칼럼의 7 이상값 처리
print(dr_ph_df[dr_ph_df['dra_01z1']==7]) # 안 나옴. 처리 필요 X


# 1-2. 연간 음주 빈도 칼럼의 7 이상값 처리
# 1-2-1. 평생 음주 여부 '2번 아니오'와 연간 음주 빈도 '8번 비해당'의 값을 갖는 row가 동일한지 확인 위해 각 인덱스 추출
dra_01z1_2 = dr_ph_df[dr_ph_df['dra_01z1']==2].index
drb_01z3_8 = dr_ph_df[dr_ph_df['drb_01z3']==8].index

# 1-2-2. 위 추출한 모든 인덱스 값이 같은지 확인
np.array_equal(dra_01z1_2, drb_01z3_8) # True

# 1-2-3. 연간 음주 빈도 8번 '비해당' 값을 1번 '최근 1년간 전혀 마시지 않았다'로 값 변경
dr_ph_df['drb_01z3'] = dr_ph_df['drb_01z3'].replace(8, 1)

# 1-2-4. 7 이상의 row 확인
print(dr_ph_df[dr_ph_df['drb_01z3']>=7]) # 2개 조회

# 1-2-5. NaN으로 대체 (NaN을 허용하지 않는 모델은 별도의 처리 필요)
dr_ph_df.loc[dr_ph_df['drb_01z3'] >= 7, 'drb_01z3'] = np.nan

Empty DataFrame
Columns: [dra_01z1, drb_01z3, drb_03z1, drb_04z1, drb_05z1, drg_01z3, pha_04z1, pha_07z1, phb_01z1, pha_11z1, churn]
Index: []
       dra_01z1  drb_01z3  drb_03z1  drb_04z1  drb_05z1  drg_01z3  pha_04z1  \
55135       1.0       7.0       7.0       7.0       8.0       7.0       0.0   
65218       1.0       7.0       7.0       7.0       8.0       7.0       0.0   

       pha_07z1  phb_01z1  pha_11z1  churn  
55135       0.0       0.0       1.0      0  
65218       0.0       0.0       1.0      1  


In [100]:
# 2 '한 번 섭취 시 음주량' ['drb_03z1']

# 2-1. 최근 1년 동안 음주를 '해본' 사람에게 한 설문이므로, 8번 비해당은 '안 한' 사람. 따라서 NaN 대신 -1 저장해 모델이 따로 학습할 수 있도록 함
# 2-1-1. 8번 비해당의 칼럼값을 -1로 대체
dr_ph_df['drb_03z1'] = dr_ph_df['drb_03z1'].replace(8, -1)

# 2-1-2. 7 이상의 row 확인
print(dr_ph_df[dr_ph_df['drb_03z1']>=7]) # 3개 조회

# 2-1-3. NaN 대체
dr_ph_df.loc[dr_ph_df['drb_03z1'] >= 7, 'drb_03z1'] = np.nan

       dra_01z1  drb_01z3  drb_03z1  drb_04z1  drb_05z1  drg_01z3  pha_04z1  \
3941        1.0       2.0       9.0       1.0       8.0       1.0       0.0   
55135       1.0       NaN       7.0       7.0       8.0       7.0       0.0   
65218       1.0       NaN       7.0       7.0       8.0       7.0       0.0   

       pha_07z1  phb_01z1  pha_11z1  churn  
3941        7.0       7.0       1.0      1  
55135       0.0       0.0       1.0      0  
65218       0.0       0.0       1.0      1  


In [101]:
# 3. '월간 폭음 경험(남)', '월간 폭음 경험(여)' ['drb_04z1', 'drb_05z1']
# 피쳐 줄이기를 위해 '월간 폭음 경험' 새 칼럼 생성해 성별에 따라 해당 칼럼값을 저장

# 3-1. 성별 칼럼 불러오기
dr_ph_df['sex'] = df['sex'] # 1 남자, 2 여자

#3-2. 월간 폭음 경험 통합
dr_ph_df['drb_binge_monthly'] = np.where(
    dr_ph_df['sex'] == 1, dr_ph_df['drb_04z1'], dr_ph_df['drb_05z1']
)

# 3-3. 불필요한 원래 칼럼, 다른 카테고리에서 처리될 성별 칼럼 제거
dr_ph_df.drop(['drb_04z1', 'drb_05z1', 'sex'], axis=1, inplace=True)


In [102]:
# 3. 월간 폭음 경험 (이어서)
# 3-2. 8비해당은 전 질문(연간 음주 빈도)에서 '최근 1년간 전혀 마시지 않았다'를 선택한 사람이므로 NaN 대신 -1로 값 대체
dr_ph_df['drb_binge_monthly'] = dr_ph_df['drb_binge_monthly'].replace(8, -1)

# 2-1-2. 7 이상의 row 확인
print(dr_ph_df[dr_ph_df['drb_binge_monthly']>=7]) # 5개 조회

# 2-1-3. NaN 대체
dr_ph_df.loc[dr_ph_df['drb_binge_monthly'] >= 7, 'drb_binge_monthly'] = np.nan

       dra_01z1  drb_01z3  drb_03z1  drg_01z3  pha_04z1  pha_07z1  phb_01z1  \
36298       1.0       4.0       2.0       9.0       0.0       2.0       0.0   
51276       1.0       6.0       4.0       3.0       0.0       5.0       7.0   
55135       1.0       NaN       NaN       7.0       0.0       0.0       0.0   
65218       1.0       NaN       NaN       7.0       0.0       0.0       0.0   
70231       1.0       5.0       3.0       1.0       0.0       0.0       7.0   

       pha_11z1  churn  drb_binge_monthly  
36298       3.0      0                9.0  
51276       6.0      0                7.0  
55135       1.0      0                7.0  
65218       1.0      1                7.0  
70231       1.0      0                9.0  


In [103]:
# 4. '절주 또는 금주계획 여부', ['drg_01z3']
# 4-1. 8비해당은 '최근 1년간 전혀 마시지 않았다'를 선택한 사람이지만, 의미 일관성을 위해 -1이 아닌 NaN으로 대체
dr_ph_df.loc[dr_ph_df['drg_01z3'] >= 7, 'drg_01z3'] = np.nan

### 신체활동 칼럼 select 과정

In [104]:
# 5. '고강도 신체활동 일수', ''중강도 신체활동 일수', '걷기 실천 일수' ['pha_04z1', 'pha_07z1', 'phb_01z1']
# 5-1. 77 이상 값 확인
dr_ph_df[ (dr_ph_df[['pha_04z1', 'pha_07z1', 'phb_01z1']] >= 77).any(axis=1) ] # 6개

# 5-2. 77 이상 값에 대해 NaN 대체
dr_ph_df.loc[dr_ph_df['pha_04z1'] >= 77, 'pha_04z1'] = np.nan
dr_ph_df.loc[dr_ph_df['pha_07z1'] >= 77, 'pha_07z1'] = np.nan
dr_ph_df.loc[dr_ph_df['phb_01z1'] >= 77, 'phb_01z1'] = np.nan

In [105]:
# 6. '최근 1주일 유연성 운동 실천', ['pha_11z1']
# 6-1. 9 이상 값 NaN 대체 (row 1개)
dr_ph_df.loc[dr_ph_df['pha_11z1'] >= 9, 'pha_11z1'] = np.nan

In [106]:
# '평생 음주 여부' 칼럼을 범주형으로(float형 -> object형) 지정
# dra_01z1 칼럼을 범주형(object)으로 변환
dr_ph_df['dra_01z1'] = dr_ph_df['dra_01z1'].astype('object')

In [107]:
dr_ph_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89822 entries, 0 to 89821
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   dra_01z1           89822 non-null  object 
 1   drb_01z3           89820 non-null  float64
 2   drb_03z1           89819 non-null  float64
 3   drg_01z3           66997 non-null  float64
 4   pha_04z1           89822 non-null  float64
 5   pha_07z1           89820 non-null  float64
 6   phb_01z1           89818 non-null  float64
 7   pha_11z1           89821 non-null  float64
 8   churn              89822 non-null  int64  
 9   drb_binge_monthly  89817 non-null  float64
dtypes: float64(8), int64(1), object(1)
memory usage: 6.9+ MB
